In [52]:
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import re
from keras.callbacks import LambdaCallback
import numpy as np
import random
import sys
import io

In [53]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras import optimizers
from keras.optimizers import RMSprop

In [54]:
def func(df,max_len_synopsis):
    fin_text=df.iloc[0]['synopsis']
    max_len_synopsis = max(max_len_synopsis, len(fin_text))
    chars_temp = set(fin_text)
    #chars.union(chars_temp)
    for review in df.iloc[0]['reviews']:
        chars_temp=chars_temp.union(set(review))
    return chars_temp,max_len_synopsis

In [55]:
from ast import literal_eval

In [56]:
chunksize = 1
fin_text=""
chars = set(fin_text)
max_len_synop=0
for chunk in pd.read_csv('preprocessed.tsv', chunksize=chunksize, delimiter='\t'):
    df= pd.DataFrame(chunk)
    df['reviews'] = df['reviews'].apply(literal_eval)
    temp=func(df,max_len_synop)
    chars = chars.union(temp[0])
    max_len_synop=max(max_len_synop,temp[1])
chars_tuple = tuple(chars)
int2char = dict(enumerate(chars_tuple))
char2int = {ch: ii for ii, ch in int2char.items()}

In [57]:
print(chars)
print(max_len_synop)

{'1', 'm', 'P', 'K', 'F', 'f', '4', '#', '7', 'B', '6', '*', "'", 'W', 'l', 'C', 'S', 'Q', '-', 'p', 'b', '8', '2', '=', '(', ' ', 'u', '"', 'T', 'J', 'ê', ',', 'R', ']', 'g', '?', 'L', '`', 'O', 'H', 'h', 'n', 't', ')', 'V', 'c', 'e', 'D', 'j', '%', 'y', '0', 'U', '~', 'k', ':', 'a', 'z', '$', '/', 'v', 'd', 'A', 'E', 'M', 'Z', 'q', 'N', '&', 's', '[', '9', '3', '\n', 'é', '+', '!', '\x96', 'X', 'r', 'w', '.', '5', ';', 'I', 'x', 'o', 'Y', 'G', 'i'}
11801


In [58]:
#df=pd.read_csv('preprocessed.tsv',delimiter='\t', nrows=3)

In [59]:
len_of_review = len(df.iloc[0]['reviews'][0])
fortyforty = (int)((len_of_review -35)/5)


In [60]:

global_review = " "
global_synop = " "

In [61]:
# max_len_synopsis=0
# max_len_reviews=0 
# no_of_reviews=0
# tot_no_of_reviews=0
# fin_text=''
# for index,row in df.iterrows():
#     max_len_synopsis = max(max_len_synopsis, len(row['synopsis']))
#     text=''.join(row['synopsis'])
#     fin_text+=' '+text
# for i in range(0, len(df)):
#     temp = df.iloc[i]['reviews']
#     no_of_reviews=0;
#     for text in temp:
#         max_len_reviews = max(max_len_reviews, len(text))
#         no_of_reviews=no_of_reviews+1
#         fin_text+=' '+text
#     tot_no_of_reviews += no_of_reviews
# print(tot_no_of_reviews)
# chars = tuple(set(fin_text))
# int2char = dict(enumerate(chars))
# char2int = {ch: ii for ii, ch in int2char.items()}
# fortyforty = (int)((max_len_reviews-35)/5)

# #print(fortyforty)
# max_len_reviews = 36+5*fortyforty
# #print(max_len_reviews)

In [62]:
def one_hot_encoder(synopsis, rev40):
#     print(max_len_synop)
    encoded = np.array([char2int[ch] for ch in synopsis])
    encoded2 = np.array([char2int[ch] for ch in rev40])
    ans = np.zeros((max_len_synop+40, len(int2char)), dtype = np.float32)
    cnt=max_len_synop-len(synopsis)
    for x in encoded:
#         print(x)
        ans[cnt][x] = 1.0
        cnt=cnt+1
    for x in encoded2:
#         print(x)
        ans[cnt][x] = 1.0
        cnt=cnt+1
    return ans

In [63]:
# def one_hot_encoder(arr, n_labels):
    
#     # Initialize the the encoded array
#     one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
#     # Fill the appropriate elements with ones
#     one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
#     # Finally reshape it to get back to the original array
#     one_hot = one_hot.reshape((*arr.shape, n_labels))
    
#     return one_hot

In [64]:
def char_encoding2arr(c):
    encoded = np.zeros(len(int2char), dtype = np.float32)
    encoded[char2int[c]]=1.0
    return encoded

In [65]:

model = Sequential()
model.add(LSTM(128, input_shape=(40+max_len_synop, len(int2char))))
model.add(Dense(len(int2char)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [66]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(global_review) - 40- 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = global_review[start_index: start_index +40]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(300):
            x_pred = np.zeros((1, 40+max_len_synop, len(int2char)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t+max_len_synop, char2int[char]] = 1.
            x_pred[0] = one_hot_encoder(global_synop, sentence)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [67]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,mode='min')

In [68]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [69]:
def ainvay():
    print(len(global_review))
    print(len(global_synop))

In [70]:
def machau(synopsis,review):
    fortyforty = (len(review) -36)//5
    Y = np.zeros((fortyforty, len(int2char)), dtype=np.float32)
    X = np.zeros((fortyforty, 40+max_len_synop, len(int2char)), dtype = np.float32)
    for k in range(40, len(review), 5):
        l = (int)((k-40)/5)
        Y[l] = char_encoding2arr(review[k])
        
    for i in range(fortyforty):
        start_index=i*5;
        X[i]=one_hot_encoder(synopsis,review[start_index:start_index+40])
    global global_review 
    global global_synop 
    global_review = review
    global_synop = synopsis
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    filepath = "weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=1, min_lr=0.001)
    callbacks = [print_callback, checkpoint, reduce_lr]
    ainvay()
#     print(len(global_review))
    model.fit(X , Y , batch_size=3 , epochs=3 ,callbacks=callbacks)
#     print(Y[0])
#     print(X[0])

In [71]:
chunksize = 1
for chunk in pd.read_csv('preprocessed.tsv', chunksize=chunksize, delimiter='\t'):
    df= pd.DataFrame(chunk)
    df['reviews'] = df['reviews'].apply(literal_eval)
    for review in df.iloc[0]['reviews']:
        machau(df.iloc[0]['synopsis'],review)
        break
    break 

1734
11801
Epoch 1/3
339/339 [==============================] - 536s 2s/step - loss: 3.3603

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "don't normally watch them. I work at a b"
don't normally watch them. I work at a b ehea o  h e he  he  he aoe  hh aeao oe  te  o  ae  e hhhe he het ee hthhhee ae hh e heo hhe eh  h  th ooaeee o e hhehe a ee oth t het teehe  aeh hhhee heee heeeotethh   ee e h ah on  hahh hhe e heeeeo he a hee e h  at  hh e o  h   a oh oe e  a e a o h  h he heeeaate et  hha  eeae hhh  aae ohaoh h h
----- diversity: 0.5
----- Generating with seed: "don't normally watch them. I work at a b"
don't normally watch them. I work at a bhh  ioso od ve ee ehh meeh aee ahhedcewthemhoeahh eahial yna  eo ne heohohttsheeh   Iehhe arehhhe natthh othiihttea enhcetetn th shh aheeh e  ti  eo aa  tih  add o aawaeae heae ooitteohresaeeorh aeoet on oehaotohahhhtht heeoeot tee hh s ate h ehesehte d hteto Kodv tdathah n s h noh eh  de het ose th
----

In [72]:
def generate_text(length, diversity,review,synopsis):
    # Get random starting text
    start_index = random.randint(0, len(review) - 40 - 1)
    generated = ''
    sentence = review[start_index: start_index + 40]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, 40+max_len_synop, len(int2char)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t+max_len_synop, char2int[char]] = 1.
            x_pred[0] = one_hot_encoder(synopsis, sentence)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

    return generated

In [73]:
print(generate_text(300, 0.2,df.iloc[0]['reviews'][0] ,df.iloc[0]['synopsis']))

on? I am not sure - almost everything the o o o oo oo aoo te   oo oo oo    oo o  e o     oo o oaao  ooo o ooe    o oo oo o o ooa e oo      o e oo o  oo e   ooe    e o o ooo o e  o  oae oo  o o o o oo o   o o o o t o o  o   aeod e ooo  o  oo oo oo o oo o   o o     o o oo ao teo  o ooo o o o oo oe  aoa  o  o o  oo  oo a o o ooo  o  o o o o 
